In [1]:
dataset_folder = "D:\\Program\\Apollo\\instance"

In [2]:
import json
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances

register_coco_instances("apollo_sample", {}, "./sample/apollo_sample.json", dataset_folder + "/train/images")

with open("./configs/apollo_meta.json",'r') as load_f:
    apollo_metadata = json.load(load_f)

MetadataCatalog.get("apollo_sample").keypoint_names = apollo_metadata['APOLLO_CAR_KEYPOINT_NAMES']
MetadataCatalog.get("apollo_sample").keypoint_flip_map = apollo_metadata['APOLLO_CAR_KEYPOINT_FLIP_MAP']
MetadataCatalog.get("apollo_sample").keypoint_connection_rules = apollo_metadata['APOLLO_CAR_KEYPOINT_CONNECTION_RULES']
MetadataCatalog.get("apollo_sample").thing_classes = ['car']

#### Self defined Transforms

In [43]:
from detectron2.data import transforms as T
from fvcore.transforms.transform import CropTransform
import torch
import numpy as np


class RandomZoom(T.Augmentation):
    def get_transform(self, image, boxes):
        h, w = image.shape[:2]
        croph = round(h*0.3)
        cropw = round(w*0.5)
        assert h >= croph and w >= cropw, "Shape computation in {} has bugs.".format(self)
        h0 = np.random.randint(h - croph + 1)
        w0 = np.random.randint(w - cropw + 1)

        return CropTransform(w0, h0, cropw, croph)

In [27]:
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper   # the default mapper
from detectron2.data import detection_utils as utils
cfg = get_cfg()
cfg.merge_from_file("./configs/Apollo-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml") 
cfg.DATASETS.TRAIN = 'apollo_sample'
cfg.SOLVER.IMS_PER_BATCH = 1
data_loader = build_detection_train_loader(cfg,
   mapper=None)

iterator = iter(data_loader)


In [44]:
import matplotlib.pyplot as plt 
batch = next(iterator)

sample = batch[0]

augs = RandomZoom()



# get the image from sample and convert from tensor to numpy
numpy_img = sample["image"].permute(1, 2, 0).cpu().detach().numpy()
numpy_img = utils.convert_image_to_rgb(numpy_img, cfg.INPUT.FORMAT)

input = T.AugInput(numpy_img, boxes=sample["instances"].gt_boxes)
transforms = augs(input)

plt.imshow(numpy_img)
plt.show()

ValueError: only one element tensors can be converted to Python scalars